In [1]:
%matplotlib inline
from pylab import *

In [2]:
from deficit_defs import *

Splikes version  0.1.4
Plasticnet version  0.1.12
macOS-15.3-arm64-arm-64bit
Base Image File: asdf/bbsk081604_all_scale2.asdf


In [3]:
import ray 
number_of_processes=8
ray.init(num_cpus=number_of_processes)

2025-02-16 15:19:59,227	INFO worker.py:1816 -- Started a local Ray instance.


Python version:,3.12.6
Ray version:,2.38.0


(pid=55961) Splikes version  0.1.4
(pid=55961) Plasticnet version  0.1.12
(pid=55961) macOS-15.3-arm64-arm-64bit
(pid=55961) Base Image File: asdf/bbsk081604_all_scale2.asdf
(run_one_bd pid=55961) Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.:task_name:run_one_bd
(run_one_bd pid=55961) Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.asdf from cache.:task_name:run_one_bd
(run_one_bd pid=55961) Using cache_images/cache_images_f6d484251bbeda8699ed0b15303a996c.

In [4]:
base='sims/2024-06-10'
if not os.path.exists(base):
    print(f"mkdir {base}")
    os.mkdir(base)

In [5]:
def deprivation_jitter(blur=[-1,-1],
            noise=[0.1,0.1],
              scale=[1,1],
              rf_size=19,
           number_of_neurons=10,
            mu_c=0,sigma_c=0,    
            mu_r=0,sigma_r=0,
           total_time=8*day,
           save_interval=1*hour,_debug=False):

    if _debug:
        total_time=1*minute
        save_interval=1*second

        
    images=[]
    for bv in blur:
        if bv<=0:
            im=pi5.filtered_images(
                                base_image_file,
                                {'type':'dog','sd1':1,'sd2':3},
                                {'type':'norm'},
                                )
        else:
            im=pi5.filtered_images(
                                    base_image_file,
                                    {'type':'blur','size':bv},
                                    {'type':'dog','sd1':1,'sd2':3},
                                    {'type':'norm'},
                                    )
        images.append(im)
                
        
        
        
    dt=200*ms      
    eta=2e-6
    pre1=pn.neurons.natural_images_with_jitter(images[0],
                                                rf_size=rf_size,
                                                time_between_patterns=dt,
                                                sigma_r=0,
                                                sigma_c=0,
                                                verbose=False)

    pre2=pn.neurons.natural_images_with_jitter(images[1],
                                                rf_size=rf_size,
                                                other_channel=pre1,
                                                time_between_patterns=dt,
                                                mu_r=mu_r,mu_c=mu_c,
                                                sigma_r=sigma_r,sigma_c=sigma_c,
                                                verbose=False)

    if scale[0]!=1:
        pre1+=pn.neurons.process.scale_shift(scale[0],0)

    if scale[1]!=1:
        pre2+=pn.neurons.process.scale_shift(scale[1],0)
    
    pre1+=pn.neurons.process.add_noise_normal(0,noise[0])
    pre2+=pn.neurons.process.add_noise_normal(0,noise[1])

    
    
    
    pre=pre1+pre2

    post=default_post(number_of_neurons)
    c=default_bcm(pre,post)
    c.eta=eta

    sim=pn.simulation(total_time)
    sim.dt=dt

    sim.monitor(post,['output'],save_interval)
    sim.monitor(c,['weights','theta'],save_interval)

    sim+=pn.grating_response(print_time=False)

    return sim,[pre,post],[c]

In [6]:
@ray.remote
def run_one_nr(params,overwrite=False,run=True,_debug=False):
    import plasticnet as pn
    count,noise1,noise2,scale1,scale2,blur1,blur2,number_of_neurons,sfname,mu_c,sigma_c=(
        params.count,params.noise1,params.noise2,
        params.scale1,params.scale2,
        params.blur1,params.blur2,
        params.number_of_neurons,params.sfname,params.mu_c,params.sigma_c)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    seq=pn.Sequence()

    t=16*day*2
    ts=1*hour

    # DEBUG
    if _debug:
        t=1*minute
        ts=1*second
    
    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[scale1,scale2],
            noise=[noise1,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    if run:
        seq.run(display_hash=False)
        pn.save(sfname,seq) 
    
    return sfname
    

In [7]:
noise_mat=linspace(0,2,21)
noise_mat

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. ])

In [8]:
from collections import namedtuple
params = namedtuple('params', 
                    ['count', 'noise1','noise2',
                     'blur1','blur2','number_of_neurons',
                     'scale1','scale2',
                     'sfname','mu_c','sigma_c'])
all_params=[]
count=0

number_of_neurons=20
noise_mat=linspace(0,2,11)
sigma_c=0
blur_mat=linspace(-1,13,15)

for noise_count,noise in enumerate(noise_mat):
    all_params.append(params(count=count,

         noise1=noise,
         noise2=noise,

         blur1=-1,
         blur2=-1,
         scale1=1,
         scale2=1,
         mu_c=0,
         sigma_c=0,
         number_of_neurons=number_of_neurons,

        sfname=f'{base}/nr %d neurons dog %d.asdf' % 
                 (number_of_neurons,noise_count),
                ))
        
    count+=1
        
for a in all_params[:5]:
    print(a)
print("[....]")
for a in all_params[-5:]:
    print(a)

print(len(all_params))


params(count=0, noise1=0.0, noise2=0.0, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/nr 20 neurons dog 0.asdf', mu_c=0, sigma_c=0)
params(count=1, noise1=0.2, noise2=0.2, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/nr 20 neurons dog 1.asdf', mu_c=0, sigma_c=0)
params(count=2, noise1=0.4, noise2=0.4, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/nr 20 neurons dog 2.asdf', mu_c=0, sigma_c=0)
params(count=3, noise1=0.6000000000000001, noise2=0.6000000000000001, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/nr 20 neurons dog 3.asdf', mu_c=0, sigma_c=0)
params(count=4, noise1=0.8, noise2=0.8, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/nr 20 neurons dog 4.asdf', mu_c=0, sigma_c=0)
[....]
params(count=6, noise1=1.2000000000000002, noise2=1.2000000000000002, blur1=-1, blur2=-1, number_of_neu

In [9]:
do_params=make_do_params(all_params)
len(do_params)

0

In [10]:
func=run_one_nr

In [11]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)
    print(sfname)

  0%|          | 0/11 [00:00<?, ?it/s]

sims/2024-06-10/nr 20 neurons dog 0.asdf
sims/2024-06-10/nr 20 neurons dog 1.asdf
sims/2024-06-10/nr 20 neurons dog 2.asdf
sims/2024-06-10/nr 20 neurons dog 3.asdf
sims/2024-06-10/nr 20 neurons dog 4.asdf
sims/2024-06-10/nr 20 neurons dog 5.asdf
sims/2024-06-10/nr 20 neurons dog 6.asdf
sims/2024-06-10/nr 20 neurons dog 7.asdf
sims/2024-06-10/nr 20 neurons dog 8.asdf
sims/2024-06-10/nr 20 neurons dog 9.asdf
sims/2024-06-10/nr 20 neurons dog 10.asdf


In [12]:
%%time
results = [func.remote(all_params[0],overwrite=True)]
sfnames=ray.get(results)

CPU times: user 163 ms, sys: 166 ms, total: 329 ms
Wall time: 5min 44s


In [13]:
real_time=5*60+ 36
print(time2str(real_time*len(do_params)/number_of_processes))

7 m, 42.00 s


In [14]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

# MD

In [10]:
@ray.remote
def run_one_md(params,overwrite=False,run=True,_debug=False):
    import plasticnet as pn
    count,noise1,noise2,scale1,scale2,blur1,blur2,number_of_neurons,sfname,mu_c,sigma_c=(
        params.count,params.noise1,params.noise2,
        params.scale1,params.scale2,
        params.blur1,params.blur2,
        params.number_of_neurons,params.sfname,params.mu_c,params.sigma_c)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    seq=pn.Sequence()

    t=16*day*2
    ts=1*hour

    # DEBUG
    if _debug:
        t=1*minute
        ts=1*second
    
    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[1,1],
            noise=[noise2,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[scale1,scale2],
            noise=[noise1,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    if run:
        seq.run(display_hash=False)
        pn.save(sfname,seq) 
    
    return sfname
    

In [11]:
from collections import namedtuple
params = namedtuple('params', 
                    ['count', 'noise1','noise2',
                     'blur1','blur2','number_of_neurons',
                     'scale1','scale2',
                     'sfname','mu_c','sigma_c'])
all_params=[]
count=0

number_of_neurons=20
noise_mat=linspace(0,2,11)
sigma_c=0
blur_mat=linspace(-1,13,15)

for noise_count,noise in enumerate(noise_mat):
    all_params.append(params(count=count,

         noise1=noise,
         noise2=0.1,

         blur1=-1,
         blur2=-1,
         scale1=0,
         scale2=1,
         mu_c=0,
         sigma_c=0,
         number_of_neurons=number_of_neurons,

        sfname=f'{base}/md %d neurons dog %d.asdf' % 
                 (number_of_neurons,noise_count),
                ))
        
    count+=1
        
for a in all_params[:5]:
    print(a)
print("[....]")
for a in all_params[-5:]:
    print(a)

print(len(all_params))


params(count=0, noise1=0.0, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=1, sfname='sims/2024-06-10/md 20 neurons dog 0.asdf', mu_c=0, sigma_c=0)
params(count=1, noise1=0.2, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=1, sfname='sims/2024-06-10/md 20 neurons dog 1.asdf', mu_c=0, sigma_c=0)
params(count=2, noise1=0.4, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=1, sfname='sims/2024-06-10/md 20 neurons dog 2.asdf', mu_c=0, sigma_c=0)
params(count=3, noise1=0.6000000000000001, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=1, sfname='sims/2024-06-10/md 20 neurons dog 3.asdf', mu_c=0, sigma_c=0)
params(count=4, noise1=0.8, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=1, sfname='sims/2024-06-10/md 20 neurons dog 4.asdf', mu_c=0, sigma_c=0)
[....]
params(count=6, noise1=1.2000000000000002, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=1, s

In [12]:
do_params=make_do_params(all_params)
print(len(do_params))
func=run_one_md

0


In [18]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)
    print(sfname)

  0%|          | 0/11 [00:00<?, ?it/s]

sims/2024-06-10/md 20 neurons dog 0.asdf
sims/2024-06-10/md 20 neurons dog 1.asdf
sims/2024-06-10/md 20 neurons dog 2.asdf
sims/2024-06-10/md 20 neurons dog 3.asdf
sims/2024-06-10/md 20 neurons dog 4.asdf
sims/2024-06-10/md 20 neurons dog 5.asdf
sims/2024-06-10/md 20 neurons dog 6.asdf
sims/2024-06-10/md 20 neurons dog 7.asdf
sims/2024-06-10/md 20 neurons dog 8.asdf
sims/2024-06-10/md 20 neurons dog 9.asdf
sims/2024-06-10/md 20 neurons dog 10.asdf


In [19]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

# BD

In [13]:
@ray.remote
def run_one_bd(params,overwrite=False,run=True,_debug=False):
    import plasticnet as pn
    count,noise1,noise2,scale1,scale2,blur1,blur2,number_of_neurons,sfname,mu_c,sigma_c=(
        params.count,params.noise1,params.noise2,
        params.scale1,params.scale2,
        params.blur1,params.blur2,
        params.number_of_neurons,params.sfname,params.mu_c,params.sigma_c)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    seq=pn.Sequence()

    t=16*day*2
    ts=1*hour

    # DEBUG
    if _debug:
        t=1*minute
        ts=1*second
    
    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[1,1],
            noise=[noise2,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[scale1,scale2],
            noise=[noise1,noise1],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    if run:
        seq.run(display_hash=False)
        pn.save(sfname,seq) 
    
    return sfname
    

In [14]:
from collections import namedtuple
params = namedtuple('params', 
                    ['count', 'noise1','noise2',
                     'blur1','blur2','number_of_neurons',
                     'scale1','scale2',
                     'sfname','mu_c','sigma_c'])
all_params=[]
count=0

number_of_neurons=20
noise_mat=linspace(0,2,11)
sigma_c=0
blur_mat=linspace(-1,13,15)

for noise_count,noise in enumerate(noise_mat):
    all_params.append(params(count=count,

         noise1=noise,
         noise2=0.1,

         blur1=-1,
         blur2=-1,
         scale1=0,
         scale2=0,
         mu_c=0,
         sigma_c=0,
         number_of_neurons=number_of_neurons,

        sfname=f'{base}/bd %d neurons dog %d.asdf' % 
                 (number_of_neurons,noise_count),
                ))
        
    count+=1
        
for a in all_params[:5]:
    print(a)
print("[....]")
for a in all_params[-5:]:
    print(a)

print(len(all_params))


do_params=make_do_params(all_params)
print(len(do_params))
func=run_one_bd

params(count=0, noise1=0.0, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=0, sfname='sims/2024-06-10/bd 20 neurons dog 0.asdf', mu_c=0, sigma_c=0)
params(count=1, noise1=0.2, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=0, sfname='sims/2024-06-10/bd 20 neurons dog 1.asdf', mu_c=0, sigma_c=0)
params(count=2, noise1=0.4, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=0, sfname='sims/2024-06-10/bd 20 neurons dog 2.asdf', mu_c=0, sigma_c=0)
params(count=3, noise1=0.6000000000000001, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=0, sfname='sims/2024-06-10/bd 20 neurons dog 3.asdf', mu_c=0, sigma_c=0)
params(count=4, noise1=0.8, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=0, sfname='sims/2024-06-10/bd 20 neurons dog 4.asdf', mu_c=0, sigma_c=0)
[....]
params(count=6, noise1=1.2000000000000002, noise2=0.1, blur1=-1, blur2=-1, number_of_neurons=20, scale1=0, scale2=0, s

In [15]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)
    print(sfname)

  0%|          | 0/11 [00:00<?, ?it/s]

sims/2024-06-10/bd 20 neurons dog 0.asdf
sims/2024-06-10/bd 20 neurons dog 1.asdf
sims/2024-06-10/bd 20 neurons dog 2.asdf
sims/2024-06-10/bd 20 neurons dog 3.asdf
sims/2024-06-10/bd 20 neurons dog 4.asdf
sims/2024-06-10/bd 20 neurons dog 5.asdf
sims/2024-06-10/bd 20 neurons dog 6.asdf
sims/2024-06-10/bd 20 neurons dog 7.asdf
sims/2024-06-10/bd 20 neurons dog 8.asdf
sims/2024-06-10/bd 20 neurons dog 9.asdf
sims/2024-06-10/bd 20 neurons dog 10.asdf


In [23]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

# RS

In [16]:
@ray.remote
def run_one_rs(params,overwrite=False,run=True,_debug=False):
    import plasticnet as pn
    count,noise1,noise2,scale1,scale2,blur1,blur2,number_of_neurons,sfname,mu_c,sigma_c=(
        params.count,params.noise1,params.noise2,
        params.scale1,params.scale2,
        params.blur1,params.blur2,
        params.number_of_neurons,params.sfname,params.mu_c,params.sigma_c)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    seq=pn.Sequence()

    t=16*day*2
    ts=1*hour

    # DEBUG
    if _debug:
        t=1*minute
        ts=1*second
    
    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[1,1],
            noise=[noise1,noise1],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[0,1],
            noise=[noise2,noise1],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[scale1,scale2],
            noise=[noise1,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    if run:
        seq.run(display_hash=False)
        pn.save(sfname,seq) 
    
    return sfname
    

In [17]:
from collections import namedtuple
params = namedtuple('params', 
                    ['count', 'noise1','noise2',
                     'blur1','blur2','number_of_neurons',
                     'scale1','scale2',
                     'sfname','mu_c','sigma_c'])
all_params=[]
count=0

number_of_neurons=20
noise_mat=linspace(0,2,11)
sigma_c=0
blur_mat=linspace(-1,13,15)

for noise_count,noise in enumerate(noise_mat):
    all_params.append(params(count=count,

         noise1=0.1,
         noise2=noise,

         blur1=-1,
         blur2=-1,
         scale1=1,
         scale2=0,
         mu_c=0,
         sigma_c=0,
         number_of_neurons=number_of_neurons,

        sfname=f'{base}/rs %d neurons dog %d.asdf' % 
                 (number_of_neurons,noise_count),
                ))
        
    count+=1
        
for a in all_params[:5]:
    print(a)
print("[....]")
for a in all_params[-5:]:
    print(a)

print(len(all_params))


do_params=make_do_params(all_params)
print(len(do_params))
func=run_one_rs

params(count=0, noise1=0.1, noise2=0.0, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=0, sfname='sims/2024-06-10/rs 20 neurons dog 0.asdf', mu_c=0, sigma_c=0)
params(count=1, noise1=0.1, noise2=0.2, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=0, sfname='sims/2024-06-10/rs 20 neurons dog 1.asdf', mu_c=0, sigma_c=0)
params(count=2, noise1=0.1, noise2=0.4, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=0, sfname='sims/2024-06-10/rs 20 neurons dog 2.asdf', mu_c=0, sigma_c=0)
params(count=3, noise1=0.1, noise2=0.6000000000000001, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=0, sfname='sims/2024-06-10/rs 20 neurons dog 3.asdf', mu_c=0, sigma_c=0)
params(count=4, noise1=0.1, noise2=0.8, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=0, sfname='sims/2024-06-10/rs 20 neurons dog 4.asdf', mu_c=0, sigma_c=0)
[....]
params(count=6, noise1=0.1, noise2=1.2000000000000002, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=0, s

In [26]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)
    print(sfname)

  0%|          | 0/11 [00:00<?, ?it/s]

sims/2024-06-10/rs 20 neurons dog 0.asdf
sims/2024-06-10/rs 20 neurons dog 1.asdf
sims/2024-06-10/rs 20 neurons dog 2.asdf
sims/2024-06-10/rs 20 neurons dog 3.asdf
sims/2024-06-10/rs 20 neurons dog 4.asdf
sims/2024-06-10/rs 20 neurons dog 5.asdf
sims/2024-06-10/rs 20 neurons dog 6.asdf
sims/2024-06-10/rs 20 neurons dog 7.asdf
sims/2024-06-10/rs 20 neurons dog 8.asdf
sims/2024-06-10/rs 20 neurons dog 9.asdf
sims/2024-06-10/rs 20 neurons dog 10.asdf


In [27]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)

# BR

In [18]:
@ray.remote
def run_one_br(params,overwrite=False,run=True,_debug=False):
    import plasticnet as pn
    count,noise1,noise2,scale1,scale2,blur1,blur2,number_of_neurons,sfname,mu_c,sigma_c=(
        params.count,params.noise1,params.noise2,
        params.scale1,params.scale2,
        params.blur1,params.blur2,
        params.number_of_neurons,params.sfname,params.mu_c,params.sigma_c)
    
    if not overwrite and os.path.exists(sfname):
        return sfname
    
    seq=pn.Sequence()

    t=16*day*2
    ts=1*hour

    # DEBUG
    if _debug:
        t=1*minute
        ts=1*second
    
    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[1,1],
            noise=[noise2,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[0,1],
            noise=[noise1,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    seq+=deprivation_jitter(blur=[blur1,blur2],
            total_time=t,
            scale=[scale1,scale2],
            noise=[noise2,noise2],
            number_of_neurons=number_of_neurons,
            mu_c=mu_c,sigma_c=sigma_c,
            save_interval=ts)

    if run:
        seq.run(display_hash=False)
        pn.save(sfname,seq) 
    
    return sfname
    

In [19]:
from collections import namedtuple
params = namedtuple('params', 
                    ['count', 'noise1','noise2',
                     'blur1','blur2','number_of_neurons',
                     'scale1','scale2',
                     'sfname','mu_c','sigma_c'])
all_params=[]
count=0

number_of_neurons=20
noise_mat=linspace(0,2,11)
sigma_c=0

for noise_count,noise in enumerate(noise_mat):
    all_params.append(params(count=count,

         noise1=noise,
         noise2=noise,

         blur1=-1,
         blur2=-1,
         scale1=1,
         scale2=1,
         mu_c=0,
         sigma_c=0,
         number_of_neurons=number_of_neurons,

        sfname=f'{base}/br %d neurons dog %d.asdf' % 
                 (number_of_neurons,noise_count),
                ))
        
    count+=1
        
for a in all_params[:5]:
    print(a)
print("[....]")
for a in all_params[-5:]:
    print(a)

print(len(all_params))


do_params=make_do_params(all_params)
print(len(do_params))
func=run_one_rs

params(count=0, noise1=0.0, noise2=0.0, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/br 20 neurons dog 0.asdf', mu_c=0, sigma_c=0)
params(count=1, noise1=0.2, noise2=0.2, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/br 20 neurons dog 1.asdf', mu_c=0, sigma_c=0)
params(count=2, noise1=0.4, noise2=0.4, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/br 20 neurons dog 2.asdf', mu_c=0, sigma_c=0)
params(count=3, noise1=0.6000000000000001, noise2=0.6000000000000001, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/br 20 neurons dog 3.asdf', mu_c=0, sigma_c=0)
params(count=4, noise1=0.8, noise2=0.8, blur1=-1, blur2=-1, number_of_neurons=20, scale1=1, scale2=1, sfname='sims/2024-06-10/br 20 neurons dog 4.asdf', mu_c=0, sigma_c=0)
[....]
params(count=6, noise1=1.2000000000000002, noise2=1.2000000000000002, blur1=-1, blur2=-1, number_of_neu

In [20]:
### premake the images
for params in tqdm(all_params):
    result=func.remote(params,run=False,overwrite=True)
    sfname=ray.get(result)
    print(sfname)

  0%|          | 0/11 [00:00<?, ?it/s]

sims/2024-06-10/br 20 neurons dog 0.asdf
sims/2024-06-10/br 20 neurons dog 1.asdf
sims/2024-06-10/br 20 neurons dog 2.asdf
sims/2024-06-10/br 20 neurons dog 3.asdf
sims/2024-06-10/br 20 neurons dog 4.asdf
sims/2024-06-10/br 20 neurons dog 5.asdf
sims/2024-06-10/br 20 neurons dog 6.asdf
sims/2024-06-10/br 20 neurons dog 7.asdf
sims/2024-06-10/br 20 neurons dog 8.asdf
sims/2024-06-10/br 20 neurons dog 9.asdf
sims/2024-06-10/br 20 neurons dog 10.asdf


In [21]:
results = [func.remote(p) for p in do_params]
sfnames=ray.get(results)